# Calchylus - Lambda calculus in Hy
<br/>
<img src="lambda.png" height=150 width=150 />
<br/>
[Lambda calculus](https://en.wikipedia.org/wiki/Lambda_calculus) is a formal system in mathematical logic for expressing computation based on function abstraction and application using variable binding and substitution.

Calchylus is a [Hy](http://docs.hylang.org) macro to write and evaluate Lambda expressions.

### Lambda macro

In [1]:
; for lambda macro these functions needs to be available at compile time
(eval-and-compile
 
  (setv comma '·)
 
  (defn drop-until [x l] 
    (do
      (try 
        (setv y (.index l x))
        (except [ValueError] (setv y -1)))
      (cut l (inc y))))

  (defn take-until [x l] 
    (do
      (try 
        (setv y (.index l x))
        (except [ValueError] (setv y (len l))))
      (cut l 0 y)))

  (defn hcdot [expr] 
    (list 
      (map (fn [x] (if (= x hy.HyCons) comma x)) expr)))
 
  (defn args [expr]
    (if (= (first expr) comma) [] (take-until comma expr))))

(defmacro 𝜆 [&rest expr]
  `(fn [~@(args expr)] ~(first (drop-until comma expr))))

; shorthand syntax: (#λ(x y · (+ x y)) 1 2)
(defsharp λ [expr] (print expr)
  `(fn [~@(args expr)] ~(first (drop-until comma expr))))

; Ÿ Application

<function _hy_anon_fn_5 at 0x000001EC67503AE8>

#### Syntax

[`𝜆` `·` `(` `)`]

#### Terms

1. Lambda variables, any literal following Hy symbol definitions are accepted, for example: `x` ,`y`, `args`, `FUNCTION` but also any unicode name like `⨐` is ok

2. Lambda abstraction, for example `(𝜆 x · x)` creates a definition of function `(fn [x] x)`

3. Lambda application , for example `((𝜆 x · x) 1)` calls for identity function with the input value 1, so the output / value of this application is 1

#### Lambda abstraction
$$\Large(𝜆 \space x · x)$$

In [2]:
(macroexpand '(𝜆 x · x))

('fn' ['x'] 'x')

#### Lambda application

$$\Large((𝜆 \space x · x)\space y)$$

#### Identity function

Function returns same value that is passed as an argument. Let's run the simple application of the identity function:

In [3]:
((𝜆 x · x) 1)

1

#### Function as an argument

Argument passed to the function can also be yet another function. Say, we want to pass the identity function to the identity function instead of static value, then appication would look this:

$$\Large((𝜆 \space x · x) \space (𝜆 \space x · x))$$

The outcome of such application is a function, thus it alone cannot evaluate to any value. Rather we need to treat it as a function part of the application and provide input for it:

In [4]:
(((𝜆 x · x) (𝜆 x · x)) 1)

1

#### Self application

One more step toward deeper nested structures is to use argument as a function and input at the same time:

In [5]:
(((𝜆 x · (x x)) (𝜆 x · x)) 1)

1

#### Currying

Restriction of the traditional Lambda Calculus is that it allows one and only one argument per function. However, there is a way to imitate multiple argument passing with a nested structure called *currying*.

In [6]:
;((λfirst.λsecond.first x) y)
(((𝜆 x · (𝜆 y · (* y x))) 2) 2)

4

#### Apply function

Further abstracting functional properties of the lambda expressions, we can implement function caller (apply func args) with the next statement:

In [7]:
(((𝜆 func · (𝜆 arg · (func arg))) (𝜆 x · x)) 2)

2

#### Constants

Whatever is the argument, return 1:

In [8]:
((𝜆 x · 2) 1)

2

In [9]:
(, ((𝜆 x · x) 1) ((𝜆 x · x) 2) ((𝜆 x · x) 3))

(1, 2, 3)

#### Make pair

$$\Large(𝜆 \space x · (𝜆 \space y · (𝜆 \space f · ((f \space x) \space y))))$$

Select first:

In [10]:
((((𝜆 x · (𝜆 y · (𝜆 f · ((f x) y)))) 1) 2) (𝜆 x · (𝜆 y · x)))

1

Select second:

In [11]:
((((𝜆 x · (𝜆 y · (𝜆 f · ((f x) y)))) 1) 2) (𝜆 x · (𝜆 y · y)))

2

#### Twiddle

Swap function argument pair.

In [12]:
; ask for the second but because arguments are swapped, the first one is returned
((((𝜆 x · (𝜆 y · (𝜆 f · ((f y) x)))) 1) 2) (𝜆 x · (𝜆 y · y)))

1

#### Infinite recursion

Produce infinite loop:

In [13]:
(try
 ; we will catch recursion error from this line, otherwise output would be messier
 ((𝜆 x · (x x)) (𝜆 x · (x x)))
 (except (e RecursionError)
   (print "Eternal loop achieved!")))

Eternal loop achieved!


#### Free and bound variables

Lambda abstractions can hold any free variables, but at the application evaluation, none of the functions should have free variables or it will cause variable definition error. For example, this is ok since we are declaring the function, not executing it:

In [14]:
(𝜆 x · (+ x y))

<function <lambda> at 0x000001EC675E3730>

But onbound variable y will become a problem on evaluation of the application:

In [15]:
(try 
  ((𝜆 x · (+ x y)) 1)
  (except (e NameError)
    (print "'y' is not defined")))

'y' is not defined


#### No arguments?!

In original Lambda Calculus functions must have one and only one argument. Lambda macro in Calchylus is not that restricted. We can omit argument for simplicity:

In [16]:
((𝜆 · 2))

2

#### Many arguments

Also multiple arguments can be passed to the lambda function althought the original specification allows one and only one argument to be passed to the Lambda function.

Support for both zero and multiple arguments simpifies the Lambda expression notation.

In [17]:
((𝜆 x y · (+ x y)) 1 2)

3

#### Booleans

In [18]:
; TRUE = select first
(((𝜆 x · (𝜆 y · x)) 1) 2)

1

In [19]:
; FALSE = select second
(((𝜆 x · (𝜆 y · y)) 1) 2)

2

#### Logic

In [20]:
; AND
(𝜆 x · (𝜆 y · ((x y) F)))

<function <lambda> at 0x000001EC675E3730>

In [21]:
; OR
(𝜆 x · (𝜆 y · ((x T) y)))

<function <lambda> at 0x000001EC675E36A8>

In [22]:
; NOT
(𝜆 x · ((x F) T))

<function <lambda> at 0x000001EC675E3950>

#### Triangular number

Summation...

In [23]:
; loop until given limit and sum up all values -> (summa 10) -> 55

#### Product

In [24]:
; loop until given limit and sum up all values -> (product 7) -> 5040

## The [MIT](http://choosealicense.com/licenses/mit/) License

Copyright (c) 2017 Marko Manninen